In [12]:
import torch 
import numpy as np
import matplotlib.pyplot as plt
import plotly
import os
from sklearn.gaussian_process import GaussianProcessRegressor

from botorch.models import SingleTaskGP
from botorch  import fit_gpytorch_model

from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf


In [13]:
#pip install plotly

In [14]:
f1_inputs = np.load("initial_inputs1.npy")
f1_outputs = np.load("initial_outputs1.npy")
f1_inputs_2 = np.load("initial_inputs2.npy")
f1_outputs_2 = np.load("initial_outputs2.npy")

In [15]:
f1_inputs = np.concatenate((f1_inputs, f1_inputs_2)) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.201607 , 0.5658698]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.785688, 0.49287]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.701023, 0.702999]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.767155, 0.801202]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.756262, 0.621414]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.721023, 0.682999]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.707689, 0.722999]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.716578, 0.705221]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.703245, 0.682999]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.609401, 0.605101]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.629401, 0.589545]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.629401, 0.616212]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.649401, 0.622878]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.631623, 0.636212]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.611623, 0.656212]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.642734, 0.616212]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.651623, 0.620656]]))) 
f1_inputs = np.concatenate((f1_inputs, np.array([[0.642734, 0.616212]]))) 

f1_outputs = np.concatenate((f1_outputs,f1_outputs_2)) 
f1_outputs = np.append(f1_outputs,-5.9027047474104416e-49)
f1_outputs = np.append(f1_outputs,1.4694233057027134e-30)
f1_outputs = np.append(f1_outputs,6.013687404741693e-08)
f1_outputs = np.append(f1_outputs,1.029025518149088e-34)
f1_outputs = np.append(f1_outputs,3.1568882670317357e-12)
f1_outputs = np.append(f1_outputs,9.594441158769377e-09)
f1_outputs = np.append(f1_outputs,4.4984959147048566e-11)
f1_outputs = np.append(f1_outputs,6.25317474883913e-10)
f1_outputs = np.append(f1_outputs,1.2990004780975896e-06)
f1_outputs = np.append(f1_outputs,0.1971687803797823)
f1_outputs = np.append(f1_outputs,0.0019282695144167755)
f1_outputs = np.append(f1_outputs,1.378994567184674)
f1_outputs = np.append(f1_outputs,0.6197596252221214)
f1_outputs = np.append(f1_outputs,1.6960217184989461)
f1_outputs = np.append(f1_outputs,0.12963516015284113)
f1_outputs = np.append(f1_outputs,0.8437573006254239)
f1_outputs = np.append(f1_outputs,0.4426566990166618)
f1_outputs = np.append(f1_outputs,0.8437573006254239)


In [16]:
variable_value = np.linspace(0.01, 0.09, 10)
n_dims = 2 
beta =0.01# 1.6262626262626263
np.random.seed(42)
train_X = torch.tensor(f1_inputs)
train_Y = torch.tensor(f1_outputs).reshape(len(f1_outputs),-1)
best_y = np.max(train_Y.numpy())
best_x = train_X[np.argmax(train_Y.numpy())]
noise_level =0.01 # 4
candidate_list = []
best_x_list = []

#train_X = train_X1[0].reshape(8,-1)
#train_Y = train_Y1[0].reshape(1,-1)

for i in range(len(variable_value)):
    #noise_level = variable_value[i]
   

    def function_1(x1,x2, noise_level=noise_level):
        result = np.sin(5 * x1+4*x2) * (1 - np.tanh(x1 ** 2) - np.tanh(x2))\
        + np.random.randn() * noise_level
        return result
    
    
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    UCB = UpperConfidenceBound(gp, beta=beta)

    bounds = torch.stack([torch.zeros(n_dims), torch.ones(n_dims)])
    candidate, _  = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=200,
    )


    x1,x2 = candidate.numpy().ravel().tolist()
    # Get the next value for the function from the script

    y_next = function_1(x1,x2)

    if y_next > best_y:
        best_y = y_next
        best_x = x1,x2


    y_next = torch.tensor(y_next).unsqueeze(-1)

    train_X = torch.cat([train_X,candidate])
    train_Y = torch.cat([train_Y,y_next.unsqueeze(-1)])
    candidate_list.append(candidate) 
    best_x_list.append(best_x)
    #print(f"Next point: {candidate} \n Next Y {y_next}  Best_y, {best_y}")
    #print(f"beta :{beta} noise_level :{noise_level}")
    #print(f"Next point: {candidate} \n Next Y {y_next}")
    #print(f"best_x:{best_x}Best_y,{best_y}")
    

In [17]:
best_x_list

[tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64),
 tensor([0.6316, 0.6362], dtype=torch.float64)]